In [ ]:
from docx import Document
import pypandoc

def docx_to_markdown(docx_path):
    doc = Document(docx_path)
    md_lines = []

    for para in doc.paragraphs:
        style = para.style.name.lower()
        text = para.text.strip()

        if not text:
            md_lines.append("")  # blank line
            continue

        # Basic heading conversion
        if "heading 1" in style:
            md_lines.append(f"# {text}")
        elif "heading 2" in style:
            md_lines.append(f"## {text}")
        elif "heading 3" in style:
            md_lines.append(f"### {text}")
        elif "heading" in style:
            md_lines.append(f"#### {text}")
        else:
            md_lines.append(text)

    return "\n".join(md_lines)

def doc_to_markdown(doc_path):
    # Convert .doc to markdown directly via pandoc
    output = pypandoc.convert_file(str(doc_path), 'md')
    return output

markdown_output = docx_to_markdown(LOCAL_SPEED_DATING_DATA_EXPLANATIONS_PATH)

# Écriture dans un fichier .md (optionnel)
with open(LOCAL_SPEED_DATING_DATA_EXPLANATIONS_MD_PATH, "w", encoding="utf-8") as f:
    f.write(markdown_output)

print("Conversion terminée. Le markdown est prêt.")

Conversion terminée. Le markdown est prêt.


In [ ]:
import subprocess
from docx import Document
from pathlib import Path
import openai

from config import (
    secrets
)

# 🔑 Ta clé API OpenAI
openai.api_key = secrets["OPENAI_API_KEY"]

def convert_doc_to_docx(doc_path):
    """Convertit un .doc en .docx via LibreOffice."""
    result = subprocess.run(
        ['libreoffice', '--headless', '--convert-to', 'docx', str(doc_path)],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    if result.returncode != 0:
        raise RuntimeError("Erreur lors de la conversion avec LibreOffice.")
    return doc_path.with_suffix('.docx')

def extract_text_from_docx(docx_path):
    """Extrait le texte d'un fichier .docx."""
    doc = Document(docx_path)
    return "\n".join(p.text.strip() for p in doc.paragraphs if p.text.strip())

def ask_openai_to_format_markdown(text, model="gpt-4"):
    """Envoie le texte à OpenAI et demande une version Markdown propre."""
    prompt = (
        "Voici le contenu extrait d’un document Word. "
        "Peux-tu le reformater en Markdown bien structuré, clair et lisible ?\n\n"
        f"{text}"
    )

    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "Tu es un assistant expert en rédaction Markdown."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4
    )

    return response['choices'][0]['message']['content']

def process_doc_to_markdown(doc_path):
    doc_path = Path(doc_path)

    # 1. Conversion si .doc
    if doc_path.suffix.lower() == ".doc":
        print("🔄 Conversion du .doc en .docx...")
        doc_path = convert_doc_to_docx(doc_path)

    # 2. Extraction du texte
    print("📄 Extraction du texte...")
    raw_text = extract_text_from_docx(doc_path)

    # 3. Appel OpenAI pour Markdown
    print("💬 Envoi à OpenAI pour reformattage...")
    markdown_text = ask_openai_to_format_markdown(raw_text)

    # 4. Sauvegarde
    output_path = doc_path.with_suffix(".md")
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(markdown_text)

    print(f"✅ Conversion terminée : Markdown enregistré dans {output_path}")

# Exemple d'utilisation
if __name__ == "__main__":
    fichier = "ton_fichier.doc"  # Remplace par ton fichier
    process_doc_to_markdown(fichier)


In [ ]:
from docx import Document
import openai
from pathlib import Path

# Configurer la clé API OpenAI
openai.api_key = "sk-..."  # remplace par ta clé API

# Étape 1 : Lire le document Word
def read_docx_text(docx_path):
    doc = Document(docx_path)
    full_text = "\n".join(para.text.strip() for para in doc.paragraphs if para.text.strip())
    return full_text

# Étape 2 : Envoi à l'API OpenAI pour formatage Markdown
def convert_to_markdown_with_openai(text):
    prompt = (
        "Tu es un assistant expert en rédaction. "
        "Transforme le texte suivant, issu d’un document Word, "
        "en une version propre et bien structurée en Markdown :\n\n"
        f"{text}"
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Tu es un expert en rédaction Markdown."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4
    )

    return response['choices'][0]['message']['content']

# Exemple d'utilisation
docx_path = Path("mon_document.docx")  # ou après conversion depuis .doc
raw_text = read_docx_text(docx_path)
markdown_text = convert_to_markdown_with_openai(raw_text)

# Sauvegarde
with open("document_markdown.md", "w", encoding="utf-8") as f:
    f.write(markdown_text)

print("✅ Le document a été adapté en Markdown avec OpenAI.")
